In [1]:
# Import Dependencies
import pandas as pd

In [2]:
#1. Import Flat CSV

file_path = 'data/pb_winning_numbers_03-18-2017.csv'

# Kaggle .csv with base powerball numbers and draw dates
pb = pd.read_csv(file_path)
pb['DrawDate']= pd.to_datetime(pb['DrawDate'])
pb_new = pb.loc[pb['DrawDate'] >= '01/01/2010']
pb_new.head()

,DrawDate,WB1,WB2,WB3,WB4,WB5,PB,PP
0,2017-03-15,30,16,41,48,53,16,3.0
1,2017-03-11,57,41,50,26,1,11,2.0
2,2017-03-08,42,23,59,46,33,4,2.0
3,2017-03-04,2,22,63,19,18,19,3.0
4,2017-03-01,16,10,52,40,55,17,10.0


In [3]:
## Texas Lottery - adding estimated jackpot
texas_url = "https://www.txlottery.org/export/sites/lottery/Games/Powerball/Winning_Numbers/index.html_1444349437.html"

texas_data = pd.read_html(texas_url)
texas_df = texas_data[0]

## clean texas lottery data
texas_df_new = texas_df.loc[(texas_df['Draw Date'] != 'First Drawing for new Powerball matrix was 10/07/2015')]
texas_df_new1 = texas_df_new.loc[(texas_df_new['Draw Date'] != 'First Drawing for new Powerball matrix was 10/07/2015.')]
texas_df_new2 = texas_df_new1.loc[(texas_df_new1['Draw Date'] != 'First Drawing for new $2 Powerball matrix was 1/18/2012.')]
texas_df_new3 = texas_df_new2.loc[(texas_df_new2['Draw Date'] >= '01/01/2010')]
texas_df_new4 = texas_df_new3.loc[(texas_df_new3['Draw Date'] <= '03/17/2017')]
texas_df_new4['Draw Date']= pd.to_datetime(texas_df_new4['Draw Date']) 
clean_texas = texas_df_new4[['Draw Date', 'Estimated Jackpot']]
clean_texas.rename(columns={'Draw Date':'DrawDate', 'Estimated Jackpot':'estimated_jackpot'}, inplace=True)
clean_texas.head()


/home/huntercash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/huntercash/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,DrawDate,estimated_jackpot
14,2019-03-16,$495 Million
15,2019-03-13,$448 Million
16,2019-03-09,$414 Million
17,2019-03-06,$381 Million
18,2019-03-02,$348 Million


In [4]:
# Merge Clean Texas DF and pb_new df

df = pb_new.merge(clean_texas, how='inner')
df.head(1)

,DrawDate,WB1,WB2,WB3,WB4,WB5,PB,PP,estimated_jackpot
0,2017-03-15,30,16,41,48,53,16,3.0,$123 Million


In [5]:
## Powerball Sales data
sales_file = "./data/powerball_sales.xlsx"
sales_df = pd.read_excel(sales_file)

# change column to numeric type
sales_df['Number of Tickets Sold']= pd.to_numeric(sales_df['Number of Tickets Sold']) 

# rename columns
sales_df.columns = ['DrawDate', 'tickets_sold', 'jackpot_millions']

## only need tickets_sold
## only need estimated jackpot
clean_sales = sales_df[['DrawDate', 'tickets_sold']]
clean_sales.head()


,DrawDate,tickets_sold
0,2019-04-27,17204152
1,2019-04-24,15122600
2,2019-04-20,16422502
3,2019-04-17,14472208
4,2019-04-13,15663006


In [6]:
# Merge Power ball data with sales data
df_powerball = df.merge(clean_sales, how='inner')
df_powerball.head(1)

,DrawDate,WB1,WB2,WB3,WB4,WB5,PB,PP,estimated_jackpot,tickets_sold
0,2017-03-15,30,16,41,48,53,16,3.0,$123 Million,14925612


In [13]:
## Portal Seven Powerball Winner Data
winning_state_url = 'http://portalseven.com/lottery/powerball_jackpot_winners.jsp'
    
winning_state = pd.read_html(winning_state_url)[2]
lotto_header = winning_state.iloc[0] 
winning_states_df = winning_state[1:]
winning_states_df.columns = lotto_header

winning_states_df = winning_states_df.drop(columns=['#'])

winning_states_df.dtypes